# Layernorm

### 思路
#### 背景和想解决的问题：batchnorm的使用有很多限制，特别是在RNN上。
1. batchnorm的效果受minibatch的样本数影响。如果batch太小，效果就不好。
2. 更大的一个限制是BN的计算方式无法直接用在RNN上。因为RNN拿到的input sequence长度不一。BN在minibatch的sample之间计算mean和variance，这时候最长的句子最后几个字在同一个batch里面只有一个sample有值。
3. BN的计算方式同样限制了它在online learning task上的应用。

#### 思路
 - 改变batchnorm的计算方式。假设一个minibatch有M个样本，每个样本的features是D维。BN给每个feature计算M个样本之间的均值和方差。而layernorm是在每个样本的内部计算D个feature之间的均值和方差。
 - <font color=blue>**BN vs. LN**</font>
   - BN的每个layer中，同一个样本的不同feature有不同的mean, variance。同一个batch的不同样本之间share共同的mean,variance
   - LN的每个layer中，同一个样本的不同feature之间share mean and variance。同一个batch的不同样本之间的mean和variance不同。

### 效果
#### 和Batchnorm相似的地方
- 虽然计算方式不同，但总的思路还是fix每层activation，即$x@w$的means和variances。
  - <font color=blue>batchnorm按列normalization。norm之后每列有相同的mean, var。</font>$$
batchnorm(z@w)=\begin{pmatrix}
 | & | &  & |\\
|  & | & ... & |\\
 | & | &  &|
\end{pmatrix}$$
  - <font color=blue>layernorm按行normalization。norm之后每行有相同的mean, var。</font>$$
layernorm(z@w)\begin{pmatrix}
-  & - & -\\
-  & - & -\\
  & ... & \\
-  & - & -\\
\end{pmatrix}
$$
- **作用**：因为non-linearity是element-wise的计算，所以从数学计算上看，不管是按列norm还是按行norm，都让non-linearity在整个training过程中拿到了分布更stable的input。所以，从forward和backward计算本身的效果上看，batchnorm能实现的他也都可以。
  1. 都能处理covariate shift
  2. 同时也都减少了梯度消失。因为可以减少梯度计算对w的scale的依赖。
  3. 起到了regularization的作用，减少了模型中使用dropout的需求
  4. 让模型可以使用saturating non-linearities，因为它能减轻网络计算大量出现saturated neurals的情况
  5. 让模型可以使用更大的learning rate，从而更快收敛。

#### 和Batchnorm不同的地方
虽然从数学公式上看，layernorm和batchnorm的计算本质是一样的。但是他们在具体的使用场景下，还是会因为适用的对象不同而不同。
 - 在Full connected network上，他们差异不大。上面的相似性分析对应的就是这个场景。
 - 在CNN上，batchnorm的效果好。layernorm的作者推测是因为CNN中，map作为input的时候，不同位置不对称，batchnorm的处理方式刚好不受这种不对称作用影响，而layernorm的计算方向会受影响。
 - LN在RNN上的效果比batchnorm好，主要是因为batchnorm本身就不适合处理RNN中那种sequence心态的数据。
 
<font color=red>*[rk's]这里的差异有时间可以进一步自己分析*</font>